In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import time

In [190]:
data = pd.read_csv('encoded.csv')

In [191]:
#data = data.loc[0:299]
data

Age  Gender  Days Abdominal Pain  Temperature  Tenderness   
0     15       1                    1         36.9           1  \
1     14       0                    2         37.0           1   
2     16       0                    0         36.9           0   
3     14       0                    1         35.7           0   
4     16       1                    7         37.9           1   
..   ...     ...                  ...          ...         ...   
419    4       1                    2         37.1           0   
420    5       0                    0         36.7           1   
421    9       0                    1         36.8           0   
422    8       0                    3         37.5           0   
423   15       0                    2         36.8           1   

     Leukocytes [10^9/l]  CRP [mg/l]  Number of Sonographies preop   
0                  11.10         6.0                             1  \
1                  21.28        30.8                             1   
2                  13.07         1.9                             1   
3                   7.95         0.0                             2   
4                  14.82        14.4                             3   
..                   ...         ...                           ...   
419                24.10        66.2                             1   
420                16.90        12.0                             1   
421                19.25        18.0                             1   
422                 9.80        56.8                             1   
423                13.63        55.5                             1   

     Preoperative Diagnosis  
0                         0  
1                         0  
2                         0  
3                         0  
4                         0  
..                      ...  
419                       1  
420                       0  
421                       0  
422                       0  
423                       0  

[424 rows x 9 columns]

In [192]:
data = data.to_numpy()
data

array([[15. ,  1. ,  1. , ...,  6. ,  1. ,  0. ],
       [14. ,  0. ,  2. , ..., 30.8,  1. ,  0. ],
       [16. ,  0. ,  0. , ...,  1.9,  1. ,  0. ],
       ...,
       [ 9. ,  0. ,  1. , ..., 18. ,  1. ,  0. ],
       [ 8. ,  0. ,  3. , ..., 56.8,  1. ,  0. ],
       [15. ,  0. ,  2. , ..., 55.5,  1. ,  0. ]])

In [75]:
# Kernel function that measures the
# similarity of two datapoints
# Returns the value of RBF kernel

def k(xi, xj, gamma):    
    return np.exp(-gamma * np.linalg.norm(xi - xj))

In [178]:
# Computes mmd^2 using list of datapoints,
# prototypes and hyperparameter gamma

from itertools import combinations

def mmd(X, prototypes, gamma):
    # Sum of protoype proximities
    spp = 0.0
    #for i in prototypes.index:
    #    for j in prototypes.index:
    #        spp += k(prototypes.loc[i], prototypes.loc[j], gamma)
    
    # Sum of prototype-datapoint proximities
    spdp = 0.0
    #for i in prototypes.index:
    #    for j in X.index:
    #        spdp += k(prototypes.loc[i], X.loc[j], gamma)
            
        
        
        
    # Sum of datapoint proximities
    sdp = 0.0
    
    t = time.time()
    for xi in X:
        for xj in X:
            sdp += k(xi, xj, gamma)
    print(time.time() - t)
    print(f'ORIG sdp: {sdp}')
    
    t = time.time()

    # Generate combinations of row indices
    cmb = list(combinations(np.arange(X.shape[0]), 2))
    
    # Extract rows based on combinations
    utm = np.sum(list(map(lambda x: k(x[0], x[1], gamma), X[cmb])))
    print(f'cmbnpsum: {utm * 2 + X.shape[0]}')
    
    print(time.time() - t)
    
    t = time.time()
    
    sdparr = [[k(xi, xj, gamma) for xj in X] for xi in X]
    sdp = np.sum(sdparr)
    print(f'npsum: {sdp}')
    
    print(time.time() - t)
    
    # Averaging proximities to compute mmd^2
    mmd2 = 1/(prototypes.shape[0]**2) * spp - 2/(prototypes.shape[0]*X.shape[0]) * spdp + 1/(X.shape[0]**2) * sdp
    
    return mmd2

In [165]:
# Adds a prototype to list prototypes
# using the datapoints from list X

def add_prototype(X, prototypes, gamma):
    min_mmd = float('+inf')
    
    # Best prototype index (lowers mmd most)
    bpi = 0
    
    # Consider each datapoint as prototype and evaluate like that
    for idx, p in enumerate(X):
        # Creating a copy of datapoints and prototypes
        Xi = X.copy()
        pi = prototypes.copy()
        
        # Removing datapoint from list of datapoints
        Xi = np.delete(Xi, idx, axis=0)
        
        # Making the removed datapoint a prototype
        pi = np.append(pi, p)
        
        # Calculating mmd of new prototype list
        mmd_i = mmd(Xi, pi, gamma)
        
        #################################
        break
        #################################
        
        # Checking if new mmd is lower
        if mmd_i < min_mmd:
            min_mmd = mmd_i
            bpi = i
    
    # Making bpith datapoint a prototype in source lists
    prototypes = np.append(prototypes, X[bpi])
    X = np.delete(X, bpi, 0)

In [87]:
# Finds m prototypes from dataset X
# Hyperparameters m and gamma

def find_prototypes(X, m=10, gamma=0.1):
    # List of chosen prototypes
    cp = np.empty((0, data.shape[1]))
    
    # While number of prototypes is under m, find another
    while len(cp) < m:
        add_prototype(data, cp, gamma)
        # print(f'Added a prototype: {cp.loc[len(cp.index) - 1]}')
        ############################
        break
    
    return cp

In [193]:
prototypes = find_prototypes(data)

0.60325026512146
ORIG sdp: 24356.4416582303
cmbnpsum: 24356.441658230906
0.45150136947631836
npsum: 24356.441658230902
0.6650454998016357


In [96]:
print(prototypes)

     Age  Gender  Days Abdominal Pain  Temperature  Tenderness   
0    NaN     NaN                  NaN          NaN         NaN  \
1    NaN     NaN                  NaN          NaN         NaN   
2    NaN     NaN                  NaN          NaN         NaN   
3    NaN     NaN                  NaN          NaN         NaN   
4    NaN     NaN                  NaN          NaN         NaN   
..   ...     ...                  ...          ...         ...   
419  NaN     NaN                  NaN          NaN         NaN   
420  NaN     NaN                  NaN          NaN         NaN   
421  NaN     NaN                  NaN          NaN         NaN   
422  NaN     NaN                  NaN          NaN         NaN   
423  NaN     NaN                  NaN          NaN         NaN   

     Leukocytes [10^9/l]  CRP [mg/l]  Number of Sonographies preop   
0                    NaN         NaN                           NaN  \
1                    NaN         NaN                           NaN 

In [20]:
time.time()

1684149707.2526317

In [128]:
import numpy as np
from itertools import combinations

# Create a sample 2D array
array = np.array([[1, 2, 3, 4, 5, 6, 7, 8, 9], [11, 12, 13, 14, 15, 16, 17, 18, 19], [21, 22, 23, 24, 25, 26, 27, 28, 29]])
print(array.shape)

# Get the number of rows
num_rows = array.shape[0]

# Generate combinations of row indices
row_indices = np.arange(num_rows)
print(row_indices)
c = list(combinations(row_indices, 2))
print(c)

# Extract rows based on combinations
result = array[c]
print(result)


(3, 9)
[0 1 2]
[(0, 1), (0, 2), (1, 2)]
[[[ 1  2  3  4  5  6  7  8  9]
  [11 12 13 14 15 16 17 18 19]]

 [[ 1  2  3  4  5  6  7  8  9]
  [21 22 23 24 25 26 27 28 29]]

 [[11 12 13 14 15 16 17 18 19]
  [21 22 23 24 25 26 27 28 29]]]


In [132]:
import numpy as np
from itertools import combinations

print(data)
print(data.shape)

# Get the number of rows
num_rows = data.shape[0]

# Generate combinations of row indices
row_indices = np.arange(num_rows)
print(row_indices)
c = list(combinations(row_indices, 2))
print(c)

# Extract rows based on combinations
result = data[c]
print(result)

[[15.    1.    1.   36.9   1.   11.1   6.    1.    0.  ]
 [14.    0.    2.   37.    1.   21.28 30.8   1.    0.  ]
 [16.    0.    0.   36.9   0.   13.07  1.9   1.    0.  ]
 [14.    0.    1.   35.7   0.    7.95  0.    2.    0.  ]
 [16.    1.    7.   37.9   1.   14.82 14.4   3.    0.  ]]
(5, 9)
[0 1 2 3 4]
[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
[[[15.    1.    1.   36.9   1.   11.1   6.    1.    0.  ]
  [14.    0.    2.   37.    1.   21.28 30.8   1.    0.  ]]

 [[15.    1.    1.   36.9   1.   11.1   6.    1.    0.  ]
  [16.    0.    0.   36.9   0.   13.07  1.9   1.    0.  ]]

 [[15.    1.    1.   36.9   1.   11.1   6.    1.    0.  ]
  [14.    0.    1.   35.7   0.    7.95  0.    2.    0.  ]]

 [[15.    1.    1.   36.9   1.   11.1   6.    1.    0.  ]
  [16.    1.    7.   37.9   1.   14.82 14.4   3.    0.  ]]

 [[14.    0.    2.   37.    1.   21.28 30.8   1.    0.  ]
  [16.    0.    0.   36.9   0.   13.07  1.9   1.    0.  ]]

 [[14.    0.    2.   37.